In [121]:
import pandas as pd
import numpy as numpy 
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

columns = ["FollowerCount", "FollowingCount", "BioLength", "Posts", "HasProfilePic", "IsPrivate", "usernameDigitCount", "usernameLength", "isFake"]
df = pd.read_csv("Account.csv", names=columns)
df.usernameDigitCount = df["usernameDigitCount"].div(df["usernameLength"], axis=0)
df = df.drop(['usernameLength', 'HasProfilePic'], axis=1)

columns2 = ["FollowerCount", "FollowingCount", "BioLength", "Posts", "IsPrivate", "usernameDigitCount", "isFake"]
df2 = pd.read_csv("social_media_train.csv", names=columns2)

df2["IsPrivate"] = df2["IsPrivate"].map({"No": 0, "Yes": 1})

#frames = [df, df2]
#df = pd.concat(frames)

display(df)

,FollowerCount,FollowingCount,BioLength,Posts,IsPrivate,usernameDigitCount,isFake
0,258,238,0,0,0,0.000000,0
1,263,482,30,29,1,0.000000,0
2,51,78,9,0,1,0.000000,0
3,297,480,22,25,1,0.222222,0
4,113,242,0,95,1,0.000000,0
...,...,...,...,...,...,...,...
1189,0,25,138,6,0,0.000000,1
1190,0,0,52,6,0,0.428571,1
1191,0,1,92,9,0,0.066667,1
1192,1,0,60,5,0,0.076923,1


In [122]:
# train, valid, test split
train, valid, test = numpy.split(df.sample(frac=1), [int(0.6 * len(df)), int(0.8 * len(df))])


/opt/homebrew/lib/python3.11/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [123]:
def scaleTheDataset(df, oversample=False):
    x = df[df.columns[:-1]].values
    y = df[df.columns[-1]].values

    scaler = StandardScaler()
    x = scaler.fit_transform(x)

    if oversample:
        ros = RandomOverSampler()
        x, y = ros.fit_resample(x, y)

    data = numpy.hstack((x, numpy.reshape(y, (len(y), 1))))

    return data, x, y

train, x_train, y_train = scaleTheDataset(train, oversample=True)
valid, x_valid, y_valid = scaleTheDataset(valid, oversample=False)
test, x_test, y_test = scaleTheDataset(test, oversample=False)


In [124]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, confusion_matrix, ConfusionMatrixDisplay

random_forest = RandomForestClassifier()

def best_rf():
    model = random_forest.fit(x_train, y_train)
    old_f1Score = f1_score(model.predict(x_test), y_test)

    for i in range(25):
     param_dist = {'n_estimators': randint(50, 500), 
              'max_depth': randint(1, 20)}
     rand_search = RandomizedSearchCV(random_forest, param_distributions = param_dist, n_iter=5, cv=5)
     rand_search.fit(x_train, y_train)
    
     new_f1Score =  f1_score(rand_search.predict(x_valid), y_valid)

     if (new_f1Score >= old_f1Score):
        model = rand_search

    return model

In [125]:
best_model = best_rf()

best_model.fit(x_train, y_train)
y_pred = best_model.predict(x_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.99      0.95      0.97       198
           1       0.82      0.98      0.89        41

    accuracy                           0.96       239
   macro avg       0.91      0.97      0.93       239
weighted avg       0.96      0.96      0.96       239



In [127]:
import pickle
pickle_out = open("classifier.pkl", "wb")
pickle.dump(best_model, pickle_out)
pickle_out.close()